In [ ]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,desc,sum,to_date,date_format,concat_ws,format_string,to_timestamp,year,when,lit,month,date_sub,current_date,max,min
from pyspark.sql import Window
import pandas as pd

# Crear una SparkSession
spark = SparkSession.builder.appName("AzureSQL").getOrCreate()

## Credenciales BD
inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/Credenciales.txt"
keys = pd.read_csv(inpath, sep = ',')
display(keys)

# Creo variables para cada fila del DataFrame que contiene las credenciales de la bd
for index, row in keys.iterrows():
    variable_name = row['key']
    variable_value = row['value']
    globals()[variable_name] = variable_value

# Configurar las propiedades de la conexión
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:1433;database={jdbc_database_datamart}"
jdbc_properties = {
    "user": user,
    "password": password,
    "driver": driver
}

# Conexion al Blob
spark.conf.set(clave_blob, access_key_blob)


#### Lectura tablas

In [ ]:
## Contacts
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Contacts/Ds_S_Contacts.parquet"
contacts = spark.read.parquet(inpath)

In [ ]:
## Ventas 2019
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/2019/Ds_S_PaymentMethods_2019.parquet/'
payments_2019 = spark.read.parquet(inpath)


## payments_2020
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/2020/Ds_S_PaymentMethods_2020.parquet/'
payments_2020 = spark.read.parquet(inpath)


## payments_2021
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/2021/Ds_S_PaymentMethods_2021.parquet/'
payments_2021 = spark.read.parquet(inpath)


## payments_2022
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/2022/Ds_S_PaymentMethods_2022.parquet/'
payments_2022 = spark.read.parquet(inpath)


## payments_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/2023/Ds_S_PaymentMethods_2023.parquet/'
payments_2023 = spark.read.parquet(inpath)



## payments_2024
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/2024/Ds_S_PaymentMethods_2024.parquet/'
payments_2024 = spark.read.parquet(inpath)


# Union años
payments = payments_2019.union(payments_2020).union(payments_2021).union(payments_2022).union(payments_2023).union(payments_2024)

payments.createOrReplaceTempView("payments_vw")

#### Arreglo para expedientes por reserva (En payments)

In [ ]:
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

# Suponiendo que tu DataFrame se llama df

# Paso 1: Filtrar los registros con t_product_id específico, es el del seguro
df_seguros = payments.filter((col("t_product_id") == "10229456543360") & (col("t_operation_type") != "RESERVATION_CANCEL"))

# Paso 2: Crear una ventana particionada por FILE_NUMBER y t_product_id, ordenada por NET_SOLD_AMT_ITX ascendente
windowSpec = Window.partitionBy("FILE_NUMBER", "t_product_id").orderBy(col("NET_SOLD_AMT_ITX").desc())

# Paso 3: Añadir columna de número de fila dentro de cada partición
df_seguros_with_row_num = df_seguros.withColumn("row_number", row_number().over(windowSpec))


# Paso 4: Filtrar para mantener solo las filas donde row_number es 1
df_seguros_filtered = df_seguros_with_row_num.filter(col("row_number") == 1).drop("row_number")


# Paso 5: Filtrar los registros que no necesitan ser procesados
df_no_seguros = payments.filter(~((col("t_product_id") == "10229456543360") & (col("t_operation_type") != "RESERVATION_CANCEL")) | col("t_product_id").isNull())

# Paso 6: Unir los DataFrames
payments = df_seguros_filtered.unionByName(df_no_seguros)

# display(payments.filter(col('FILE_NUMBER')=='16367759'))
# a=df_filtered.filter((col('FILE_NUMBER').isin('16367759','16642411','17550029','17835393','17859550'))&(col('t_product_id')=='10229456543360'))
# display(a)

In [ ]:
## TABLA SALES
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/Completo_ventas/Ds_G_sales.parquet'
sales_ = spark.read.parquet(inpath)
# tmp = sales.select('T_OPERATION_KIND').distinct() 
# display(tmp.limit(10))

# Hfee 
# hfee = sales_.filter(col('T_OPERATION_KIND') == 'DELIVERY_OVERHEADS')


In [ ]:
# display(sales.limit(10))

In [ ]:
tmp = sales_.filter(col('FILE_NUMBER') == '10813033') #DELIVERY_OVERHEADS
#tmp = sales_.filter(col('T_OPERATION_KIND').like('%DELIVERY_OVERHEADS%')) #DELIVERY_OVERHEADS
display(tmp.limit(10))

ORGANIZATION,T_ORGANIZ_ID,PRODUCT_FAMILY,PRODUCT,INVOICE_CONTACT_FMT_NAME,T_PRODUCT_ID,LICENCE_NUMBER,PRODUCT_STATE,SITE,SEASON,TOPIC,SUB_TOPIC,REFERENCE_DATE,ORDER_DATE_MM_YY,PRODUCT_DATE_TIME,PRODUCT_TIME,FILE_NUMBER,T_ORDER_CONTACT_ID,PROMOTION,AUDIENCE_CATEGORY,AUDIENCE_SUB_CATEGORY,LOGICAL_SEAT_CATEGORY,PERFORMANCE_QUOTA,SALES_CHANNEL,SALES_CHANNEL_N,SALES_CHANNEL_TYPE,UNIT_AMT_ITX,NET_SOLD_T_QTY,NET_SOLD_C_QTY,NET_SOLD_TKT_AMT_ITX,TEAM_1,TEAM_2,ORDER_NUMBER,CHARGE_AMT_ITX,ORDER_CREATION_DATE,ORDER_CONTACT_NUMBER,ORDER_DATE,PRODUCT_DATE,CHARGE_AMT_ETX,total_unit_amt_itx,DN_QUOTA,T_INVOICE_CONTACT_ID,File_state,t_file_state,ORDER_QUANTITY,T_OPERATION_TYPE,PRODUCT_CODE,t_operation_kind,parent_product,INSURED,INSURED_AMT_ITX,T_PERFORMANCE_ID,T_SITE_ID,NET_SOLD_AMT_ITX,PRODUCT_EXTERNAL_NAME,T_SEASON_ID,ACTIVITY,T_ACTIVITY_ID,T_PRODUCT_FAMILY_ID,PRODUCT_TYPE,T_PRODUCT_TYPE_ID,T_FAMILY_TYPE,T_FAMILY_SUB_TYPE,T_SHIPMENTFEE_ID,TOWN,TOPIC_KEY,TOPIC_N,SUB_TOPIC_N,INSURED_AMT_ETX,T_OPERATION_ID,ORDER_CONTACT_ADDRESS,ORDER_CONTACT_COUNTRY,ORDER_CONTACT_TOWN,ORDER_CONTACT_PRIMARY_PHONE,ORDER_CONTACT_EMAIL,Fecha_evento,Date_evento,Hora_compra,Hora_de_compra,Hora_evento,Key_evento,Key_handling,TOWN_N,Key_aforo,Nuevo_aforo,VENUE_N,Tipo_venue,TEAM_2_N,NIT,Contrato,MATCH_DAY,PRODUCT_N,TEMA_MERCADEO,SUB_TEMA_MERCADEO,Hfee,TOPIC_PPTO
Tren de la Sabana,101375831000,Evento,RECORRIDO ZIPAQUIRA - 2023,TURISTREN S.A.S ( 4590894 ),10228866549538,null,En ejecución,Tren Turistico,UNICA,11. PARQUES Y FAMILIARES,TREN DE LA SABANA,2023-03-23T00:00:00Z,03.2023,2023-03-25T08:20:00Z,08:20,10813033,10229076923830,null,Contacto ocasional,Pasajero (Adulto),TIQUETE,756.0,Tuboleta.com - ALL,Web,Internet gran público,70000.0,3.0,0.0,210000.0,null,null,18235156,18000.0,2023-03-23T21:24:23Z,8212647,23.03.2023,25.03.23,15126.0,210000.0,702.0,1.01431452368E11,Pagado,SOLDED,3,SALE,BTREN23,SINGLE_ENTRY,null,T,12051.0,10228984231312,1.01359979052E11,210000.0,RECORRIDO ZIPAQUIRA,1.01375877965E11,Eventos 2023,1.0228596429782E13,16059.0,null,null,SIMPLE_TICKET,SIMPLE_TICKET,null,BOGOTA,11. PARQUES Y FAMILIARES|TREN DE LA SABANA,RECORRIDOS,TURISMO,10127.0,10229077290291,Carrera 89 22b 96 R406 111111 BOGOTÁ COLOMBIA,Colombia,BOGOTÁ,57-3003850222,p_pedreros@hotmail.com,2023-03-25T08:20:00Z,2023-03-25,21:24:23,2023-03-23T21:24:23Z,08:20:00,10228866549538|10228984231312,10813033|10228866549538|10228984231312,BOGOTA,10228866549538|10228984231312|TIQUETE,702.0,Tren Turistico,null,null,800.181.743-1,120,null,RECORRIDO ZIPAQUIRA - 2023,Familia,Turismo,8000.0,Parques y familiares
Tuboleta.com,16261,null,SERVICIO IVA INCLUIDO,null,,null,null,null,null,null,null,2023-03-23T00:00:00Z,03.2023,null,null,10813033,10229076923830,null,null,null,null,0.0,Tuboleta.com - ALL,Web,Internet gran público,6000.0,3.0,0.0,18000.0,null,null,18235156,null,2023-03-23T21:24:23Z,8212647,23.03.2023,null,null,18000.0,0.0,null,Pagado,SOLDED,3,SALE,null,CHARGES,null,F,0.0,,null,18000.0,null,null,null,null,null,null,null,null,null,null,null,,SIN DATO,SIN DATO,0.0,10229077290292,Carrera 89 22b 96 R406 111111 BOGOTÁ COLOMBIA,Colombia,BOGOTÁ,57-3003850222,p_pedreros@hotmail.com,null,null,21:24:23,2023-03-23T21:24:23Z,null,|,10813033||,null,|,null,null,null,null,null,null,null,SERVICIO IVA INCLUIDO,null,null,0.0,null
Tuboleta.com,16261,null,E-TICKET - MOBILE - $8000,null,,null,null,null,null,null,null,2023-03-23T00:00:00Z,03.2023,null,null,10813033,10229076923830,null,null,null,null,0.0,Tuboleta.com - ALL,Web,Internet gran público,8000.0,1.0,0.0,8000.0,null,null,18235156,null,2023-03-23T21:24:23Z,8212647,23.03.2023,null,null,8000.0,0.0,null,Pagado,SOLDED,1,SALE,null,DELIVERY_OVERHEADS,null,F,0.0,,null,8000.0,null,null,null,null,null,null,null,null,null,1.01322856703E11,null,,SIN DATO,SIN DATO,0.0,10229077301697,Carrera 89 22b 96 R406 111111 BOGOTÁ COLOMBIA,Colombia,BOGOTÁ,57-3003850222,p_pedreros@hotmail.com,null,null,21:24:23,2023-03-23T21:24:23Z,null,|,10813033||,null,|,null,null,null,null,null,null,null,E-TIC

In [ ]:
# Sales del organismo Tren de la Sabana
sales = sales_.withColumn('REFERENCE_DATE', to_date('REFERENCE_DATE')).\
    filter((sales_['T_ORGANIZ_ID']=='101375831000') &
           (sales_['REFERENCE_DATE'] >= '2022-08-01'))

print(sales.count())

# Handling Fee
exp = sales.select('FILE_NUMBER').distinct().collect()
#display(tmp.limit(10))

exp = [row.FILE_NUMBER for row in exp]

hfee = sales_.filter((col('FILE_NUMBER').isin(exp)) &
                     (col('T_OPERATION_KIND').isin('DELIVERY_OVERHEADS','ORDER_OVERHEADS')))

90161


In [ ]:
display(hfee.limit(10))

ORGANIZATION,T_ORGANIZ_ID,PRODUCT_FAMILY,PRODUCT,INVOICE_CONTACT_FMT_NAME,T_PRODUCT_ID,LICENCE_NUMBER,PRODUCT_STATE,SITE,SEASON,TOPIC,SUB_TOPIC,REFERENCE_DATE,ORDER_DATE_MM_YY,PRODUCT_DATE_TIME,PRODUCT_TIME,FILE_NUMBER,T_ORDER_CONTACT_ID,PROMOTION,AUDIENCE_CATEGORY,AUDIENCE_SUB_CATEGORY,LOGICAL_SEAT_CATEGORY,PERFORMANCE_QUOTA,SALES_CHANNEL,SALES_CHANNEL_N,SALES_CHANNEL_TYPE,UNIT_AMT_ITX,NET_SOLD_T_QTY,NET_SOLD_C_QTY,NET_SOLD_TKT_AMT_ITX,TEAM_1,TEAM_2,ORDER_NUMBER,CHARGE_AMT_ITX,ORDER_CREATION_DATE,ORDER_CONTACT_NUMBER,ORDER_DATE,PRODUCT_DATE,CHARGE_AMT_ETX,total_unit_amt_itx,DN_QUOTA,T_INVOICE_CONTACT_ID,File_state,t_file_state,ORDER_QUANTITY,T_OPERATION_TYPE,PRODUCT_CODE,t_operation_kind,parent_product,INSURED,INSURED_AMT_ITX,T_PERFORMANCE_ID,T_SITE_ID,NET_SOLD_AMT_ITX,PRODUCT_EXTERNAL_NAME,T_SEASON_ID,ACTIVITY,T_ACTIVITY_ID,T_PRODUCT_FAMILY_ID,PRODUCT_TYPE,T_PRODUCT_TYPE_ID,T_FAMILY_TYPE,T_FAMILY_SUB_TYPE,T_SHIPMENTFEE_ID,TOWN,TOPIC_KEY,TOPIC_N,SUB_TOPIC_N,INSURED_AMT_ETX,T_OPERATION_ID,ORDER_CONTACT_ADDRESS,ORDER_CONTACT_COUNTRY,ORDER_CONTACT_TOWN,ORDER_CONTACT_PRIMARY_PHONE,ORDER_CONTACT_EMAIL,Fecha_evento,Date_evento,Hora_compra,Hora_de_compra,Hora_evento,Key_evento,Key_handling,TOWN_N,Key_aforo,Nuevo_aforo,VENUE_N,Tipo_venue,TEAM_2_N,NIT,Contrato,MATCH_DAY,PRODUCT_N,TEMA_MERCADEO,SUB_TEMA_MERCADEO,Hfee,TOPIC_PPTO
Tuboleta.com,16261,null,E-TICKET - MOBILE - $8000,null,,null,null,null,null,null,null,2023-03-23T00:00:00Z,03.2023,null,null,10813033,10229076923830,null,null,null,null,0.0,Tuboleta.com - ALL,Web,Internet gran público,8000.0,1.0,0.0,8000.0,null,null,18235156,null,2023-03-23T21:24:23Z,8212647,23.03.2023,null,null,8000.0,0.0,null,Pagado,SOLDED,1,SALE,null,DELIVERY_OVERHEADS,null,F,0.0,,null,8000.0,null,null,null,null,null,null,null,null,null,1.01322856703E11,null,,SIN DATO,SIN DATO,0.0,10229077301697,Carrera 89 22b 96 R406 111111 BOGOTÁ COLOMBIA,Colombia,BOGOTÁ,57-3003850222,p_pedreros@hotmail.com,null,null,21:24:23,2023-03-23T21:24:23Z,null,|,10813033||,null,|,null,null,null,null,null,null,null,E-TICKET - MOBILE - $8000,null,null,0.0,null
Tuboleta.com,16261,null,E-TICKET - MOBILE - $8000,null,,null,null,null,null,null,null,2023-04-04T00:00:00Z,04.2023,null,null,10986534,101329934940,null,null,null,null,0.0,Tuboleta.com - ALL,Web,Internet gran público,8000.0,1.0,0.0,8000.0,null,null,18456975,null,2023-04-04T21:15:41Z,29449,04.04.2023,null,null,8000.0,0.0,null,Pagado,SOLDED,1,SALE,null,DELIVERY_OVERHEADS,null,F,0.0,,null,8000.0,null,null,null,null,null,null,null,null,null,1.01322856703E11,null,,SIN DATO,SIN DATO,0.0,10229102733177,transversal 21 No 25 d 22 la pradera dosquebradas piso 2 111111 DOSQUEBRADAS COLOMBIA,Colombia,DOSQUEBRADAS,57-3116103985,osvaldo080@gmail.com,null,null,21:15:41,2023-04-04T21:15:41Z,null,|,10986534||,null,|,null,null,null,null,null,null,null,E-TICKET - MOBILE - $8000,null,null,0.0,null
Tuboleta.com,16261,null,E-TICKET - MOBILE - $8000,null,,null,null,null,null,null,null,2023-04-05T00:00:00Z,04.2023,null,null,10994750,10228648961642,null,null,null,null,0.0,Tuboleta.com - ALL,Web,Internet gran público,8000.0,1.0,0.0,8000.0,null,null,18466641,null,2023-04-05T21:30:54Z,7360992,05.04.2023,null,null,8000.0,0.0,null,Pagado,SOLDED,1,SALE,null,DELIVERY_OVERHEADS,null,F,0.0,,null,8000.0,null,null,null,null,null,null,null,null,null,1.01322856703E11,null,,SIN DATO,SIN DATO,0.0,10229104922697,Calle 52a 9-52 Chapinero 111111 BOGOTA COLOMBIA,Colombia,BOGOTA,57-3202261030,de.vargas356@gmail.com,null,null,21:30:54,2023-04-05T21:30:54Z,null,|,10994750||,null,|,null,null,null,null,null,null,null,E-TICKET - MOBILE - $8000,null,null,0.0,null
Tuboleta.com,16261,null,E-TICKET - MOBILE - $8000,null,,null,null,null,null,null,null,2023-04-18T00:00:00Z,04.2023,null,null,11153724,10229128326849,null,null,null,null,0.0,Tuboleta.com - ALL,Web,Internet gran público,8000.0,1.0,0.0,8000.0,null,null,18688592,null,2023-04-18T14:57:45Z,8282198,18.04.2023,null,null,8000.0,0.0,null,Pagado,SOLDED,1,SALE,null,DELIVERY_OVERHEAD

In [ ]:
# tmp = sales.filter(col('PRODUCT_CODE') == 'CAN_INS') #DELIVERY_OVERHEADS
# #tmp = sales_.filter(col('T_OPERATION_KIND').like('%DELIVERY_OVERHEADS%')) #DELIVERY_OVERHEADS
# display(tmp.limit(10))

In [ ]:
#tmp = sales.filter(col('FILE_NUMBER') == '11004473') #DELIVERY_OVERHEADS
#tmp = sales.filter(col('T_OPERATION_KIND').like('%DELIVERY_OVERHEADS%')) #DELIVERY_OVERHEADS
# display(tmp.limit(10))

#### Arreglo seguro

In [ ]:
import pandas as pd
from pyspark.sql.functions import col, concat, lit


# Filtro para el seguro de asistencia cuando son ventas
sale_df_70 = sales.filter((col('product_code') == 'CAN_INS') & (col('t_operation_type') == 'SALE')) \
            .withColumn('product', concat(col('product'), lit('_S_70'))) \
            .withColumn('PRODUCT_CODE',concat(col('Product_code'), lit('_70'))) \
            .withColumn('NET_SOLD_TKT_AMT_ITX',col('NET_SOLD_TKT_AMT_ITX')*0.7) \
            .withColumn('NET_SOLD_AMT_ITX',col('NET_SOLD_AMT_ITX')*0.7)

sale_df_30 = sales.filter((col('product_code') == 'CAN_INS') & (col('t_operation_type') == 'SALE')) \
            .withColumn('product', concat(col('product'), lit('_S_30')))\
            .withColumn('PRODUCT_CODE',concat(col('Product_code'), lit('_30')))\
            .withColumn('NET_SOLD_TKT_AMT_ITX',col('NET_SOLD_TKT_AMT_ITX')*0.3)\
            .withColumn('NET_SOLD_AMT_ITX',col('NET_SOLD_AMT_ITX')*0.3)

# Filtro para el seguro de asistencia cuando son devoluciones
refund_df_70 = sales.filter((col('product_code') == 'CAN_INS') & (col('t_operation_type') == 'REFUND')) \
              .withColumn('product', concat(col('product'), lit('_R_70')))\
            .withColumn('PRODUCT_CODE',concat(col('Product_code'), lit('_70')))\
            .withColumn('NET_SOLD_TKT_AMT_ITX',col('NET_SOLD_TKT_AMT_ITX')*0.7)\
            .withColumn('NET_SOLD_AMT_ITX',col('NET_SOLD_AMT_ITX')*0.7)

refund_df_30 = sales.filter((col('product_code') == 'CAN_INS') & (col('t_operation_type') == 'REFUND')) \
              .withColumn('product', concat(col('product'), lit('_R_30')))\
            .withColumn('PRODUCT_CODE',concat(col('Product_code'), lit('_30')))\
            .withColumn('NET_SOLD_TKT_AMT_ITX',col('NET_SOLD_TKT_AMT_ITX')*0.3)\
            .withColumn('NET_SOLD_AMT_ITX',col('NET_SOLD_AMT_ITX')*0.3)
              



# Unir los DataFrames originales con los duplicados
sales = sales.union(sale_df_70).union(sale_df_30).union(refund_df_70).union(refund_df_30).union(hfee)

# tmp = sales.select('t_operation_kind').distinct() 
# display(tmp.limit(10))

#sales = sales.filter((col('PRODUCT_CODE')!='CAN_INS'))


sales = sales.filter((col('PRODUCT_CODE')!='CAN_INS') |
                     (col('t_operation_kind').isin('DELIVERY_OVERHEADS','ORDER_OVERHEADS')))

print(sales.count())

124469


In [ ]:
from pyspark.sql.functions import year, month
# Convertir la columna REFERENCE_DATE a tipo de dato timestamp
sales = sales.withColumn('REFERENCE_DATE', col('REFERENCE_DATE').cast('timestamp'))

# Crear nuevas columnas con el año y el mes
sales = sales.withColumn('year', year('REFERENCE_DATE'))
sales = sales.withColumn('month', month('REFERENCE_DATE'))

In [ ]:
tmp = sales.select('t_operation_kind').distinct() 
display(tmp.limit(10))

t_operation_kind
COMPOSED_PRODUCT
PRODUCT_COMPOSITION
SINGLE_ENTRY
SIMPLE_PRODUCT
ORDER_OVERHEADS
DELIVERY_OVERHEADS


#### Query

In [ ]:
sales.createOrReplaceTempView("d_sales_list_sales_v1_0")
payments.createOrReplaceTempView("D_SALES_PROD_PAYMENT_V1_0")
contacts.createOrReplaceTempView("D_CONTACT_LIST_V1_0")

In [ ]:
query= """
SELECT
    CASE
        WHEN TRIM(CL.ID_NUMBER) IS NULL OR TRIM(CL.ID_NUMBER) = '' THEN '222222222222'
        WHEN (LENGTH(CL.ID_NUMBER) > 9 AND INSTR(CL.ID_NUMBER, '9') > 0) THEN '222222222222'
        WHEN REGEXP_LIKE(CL.ID_NUMBER, '^[a-zA-Z]*[a-zA-Z][a-zA-Z0-9-]*$')
            AND CL.ID_NUMBER NOT LIKE 'CC%' 
            AND CL.ID_NUMBER NOT LIKE 'CE%' THEN '222222222222'
        WHEN CL.ID_NUMBER LIKE 'CC%' THEN SUBSTR(CL.ID_NUMBER, 3)
        WHEN CL.ID_NUMBER LIKE 'CE%' THEN SUBSTR(CL.ID_NUMBER, 3)
        ELSE CL.ID_NUMBER
    END AS IDENTIFICACION,
CASE   
         WHEN LENGTH(CL.ID_NUMBER) = 9 
            AND INSTR(CL.ID_NUMBER, '9') > 0
            AND CL.ID_NUMBER NOT LIKE 'CC%' 
            AND CL.ID_NUMBER NOT LIKE 'CE%'
            AND REGEXP_LIKE(CL.ID_NUMBER, '^[0-9]+$')
         THEN '31'
        WHEN LENGTH(CL.ID_NUMBER) = 8 AND INSTR(CL.ID_NUMBER, '8') > 0 
            AND CL.ID_NUMBER NOT LIKE 'CC%' 
            AND CL.ID_NUMBER NOT LIKE 'CE%'
        THEN '31'
        WHEN LENGTH(CL.ID_NUMBER) = 7 OR LENGTH(CL.ID_NUMBER) = 8 
            AND CL.ID_NUMBER NOT LIKE 'CC%' 
            AND CL.ID_NUMBER NOT LIKE 'CE%'
        THEN '13'
        WHEN  
            LENGTH(CL.ID_NUMBER) > 9 
            AND CL.ID_NUMBER NOT LIKE 'CC%' 
            AND CL.ID_NUMBER NOT LIKE 'CE%'
            AND CL.ID_NUMBER NOT LIKE '222222222222%'
            AND REGEXP_LIKE(CL.ID_NUMBER, '^[a-zA-Z0-9]+$')
        THEN '13'
        WHEN LENGTH(CL.ID_NUMBER) = 6 
            AND CL.ID_NUMBER NOT LIKE 'CC%' 
            AND CL.ID_NUMBER NOT LIKE 'CE%'
        THEN '42'  
        WHEN CL.ID_NUMBER LIKE 'CE%' THEN '42'
        WHEN CL.ID_NUMBER LIKE 'CC%' THEN '13'
        ELSE '43'
END AS Tipo_identificacion,
CASE
    
        WHEN LENGTH(CL.ID_NUMBER) = 9 AND INSTR(CL.ID_NUMBER, '9') > 0 THEN 'Empresa' 
        WHEN LENGTH(CL.ID_NUMBER) = 9 AND INSTR(CL.ID_NUMBER, '8') > 0 THEN 'Empresa'
        WHEN LENGTH(CL.ID_NUMBER) = 7 THEN 'Es persona'
        WHEN LENGTH(CL.ID_NUMBER) = 8 THEN 'Es persona'
        WHEN LENGTH(CL.ID_NUMBER) > 9 THEN 'Es persona'
        WHEN LENGTH(CL.ID_NUMBER) = 6 THEN 'Es persona'
        ELSE 'Es persona'
END AS Tipo,
CASE    
    WHEN LENGTH(CL.ID_NUMBER) = 9 AND INSTR(CL.ID_NUMBER, '9') > 0 THEN CL.FIRSTNAME 
    WHEN LENGTH(CL.ID_NUMBER) = 9 AND INSTR(CL.ID_NUMBER, '8') > 0 THEN CL.FIRSTNAME
    ELSE ' '
END AS RAZON_SOCIAL,  
CASE
    
     WHEN LENGTH(CL.ID_NUMBER) = 9 AND INSTR(CL.ID_NUMBER, '9') > 0 THEN '' 
     WHEN LENGTH(CL.ID_NUMBER) = 9 AND INSTR(CL.ID_NUMBER, '8') > 0 THEN ''
     ELSE CL.FIRSTNAME
END AS PRIMER_NOMBRE,    
CL.LASTNAME,
 SUBSTR(SLS.ORDER_CONTACT_ADDRESS,0,13) AS Direccion,

CASE
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Brasil%' THEN 'Br'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Canadá%' THEN 'Ca'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Chile%' THEN 'Cl'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Colombia%' THEN 'Co'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Costa Rica%' THEN 'Cr'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Ecuador%' THEN 'Ec'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Estados Unidos%' THEN 'Us'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Francia%' THEN 'Fr'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Honduras%' THEN 'Hn'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%México%' THEN 'Mx'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Panamá%' THEN 'Pa'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Perú%' THEN 'Pe'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Reino Unido%' THEN 'Gb'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Uruguay%' THEN 'Uy'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Venezuela%' THEN 'Ve'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Alemania%' THEN 'De'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Argentina%' THEN 'Ar'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Bolivia%' THEN 'Bo'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%España%' THEN 'Es'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Guatemala%' THEN 'Gt'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Italia%' THEN 'It'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Noruega%' THEN 'No'
    WHEN (SLS.ORDER_CONTACT_COUNTRY) LIKE '%Salvador%' THEN 'Sv'
    ELSE 'CO' 
END AS CODIGO_PAIS,

CASE
    WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%RÍO DE JANEIRO%' THEN '18'
    WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%AGUACHICA%' THEN '20'
    WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%AGUAZUL%' THEN '85'
    WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%AIPE%' THEN '41'
    WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ANAPOIMA%' THEN '25'
    WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%AQUITANIA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ARAUCA%' THEN '81'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ARCABUCO%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ARMENIA%' THEN '05'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BARBOSA%' THEN '05'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BARICHARA%' THEN '68'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BARRANCABERMEJA%' THEN '68'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BARRANQUILLA%' THEN '08'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BELLO%' THEN '05'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BOGOTÁ%' THEN '11'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BUCARAMANGA%' THEN '68'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CAJICÁ%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CALI%' THEN '76'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CAMPOALEGRE%' THEN '41'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CARTAGENA%' THEN '13'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CERINZA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CHÍA%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CHIQUINQUIRÁ%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CIRCASIA%' THEN '63'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CORRALES%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%COTA%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%DUITAMA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%EL DOVIO%' THEN '76'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ENVIGADO%' THEN '05'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ESPINAL%' THEN '73'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FACATATIVÁ%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FIRAVITOBA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FLORENCIA%' THEN '18'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FLORESTA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FLORIDABLANCA%' THEN '68'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FUNZA%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FUSAGASUGÁ%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GAMEZA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GARAGOA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GIRARDOT%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GIRARDOTA%' THEN '05'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GIRÓN%' THEN '68'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GUASCA%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%HONDA%' THEN '73'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%IBAGUÉ%' THEN '73'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ITAGUI%' THEN '05'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%IZA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA CALERA%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA CEJA%' THEN '05'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA DORADA%' THEN '17'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA ESTRELLA%' THEN '05'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA PEÑA%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA TEBAIDA%' THEN '63'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA VEGA%' THEN '19'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MADRID%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MANIZALES%' THEN '17'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MANTA%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MEDELLÍN%' THEN '05'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MEDINA%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MELGAR%' THEN '73'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MIRAFLORES%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MONGUA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MONTERREY%' THEN '85'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MOSQUERA%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MOTAVITA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%NEIVA%' THEN '41'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%NOBSA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%OIBA%' THEN '68'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PAIPA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PALMIRA%' THEN '76'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PAMPLONA%' THEN '54'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PASCA%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PASTO%' THEN '52'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PAUNA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PAZ DE ARIPORO%' THEN '85'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PEREIRA%' THEN '66'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PESCA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PIEDECUESTA%' THEN '68'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PITALITO%' THEN '41'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PORE%' THEN '85'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%RESTREPO%' THEN '50'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%RIOHACHA%' THEN '44'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%RIONEGRO%' THEN '05'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ROVIRA%' THEN '73'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SABANETA%' THEN '05'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SALITRE%' THEN '14'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SAN ALBERTO%' THEN '20'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SAN BERNARDO%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SAN GIL%' THEN '68'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SAN LUIS DE PALENQUE%' THEN '85'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SANTA FE%' THEN '31'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SANTA MARTA%' THEN '47'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SANTA ROSA DE VITERBO%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SANTANA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SANTANDER DE QUILICHAO%' THEN '19'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SIBATÉ%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SINCELEJO%' THEN '70'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SOACHA%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SOCHA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SOCORRO%' THEN '68'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SOGAMOSO%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SOPÓ%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SUAITA%' THEN '68'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SUCRE%' THEN '19'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SUESCA%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TABIO%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TAME%' THEN '81'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TENJO%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TIBASOSA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TOCA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TOTA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TRINIDAD%' THEN '85'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TUNJA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TUTA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%VALLEDUPAR%' THEN '20'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%VENTAQUEMADA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%VILLA DE LEYVA%' THEN '15'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%VILLA DE SAN DIEGO DE UBATE%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%VILLA DEL ROSARIO%' THEN '54'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%YOPAL%' THEN '85'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%YUMBO%' THEN '76'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ZIPAQUIRÁ%' THEN '25'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GUAYAQUIL%' THEN '14'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PARÍS%' THEN '76'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SAN PEDRO SULA%' THEN '02'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CIUDAD DE MÉXICO%' THEN '07'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GUADALAJARA%' THEN '14'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LIMA%' THEN '47'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LONDRES%' THEN '01'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MONTEVIDEO%' THEN '10'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BARQUISIMETO%' THEN '13'
        ELSE '11'
END AS CODIGO_ESTADO,

CASE
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%RíO DE JANEIRO%' THEN '1801'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%AGUACHICA%' THEN '20011'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%AGUAZUL%' THEN '85010'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%AIPE%' THEN '41016'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ANAPOIMA%' THEN '25035'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%AQUITANIA%' THEN '15047'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ARAUCA%' THEN '81001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ARCABUCO%' THEN '15051'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ARMENIA%' THEN '05059'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BARBOSA%' THEN '05079'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BARICHARA%' THEN '68079'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BARRANCABERMEJA%' THEN '68081'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BARRANQUILLA%' THEN '08001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BELLO%' THEN '05088'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BOGOTÁ%' THEN '11001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%BUCARAMANGA%' THEN '68001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CAJICÁ%' THEN '25126'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CALI%' THEN '76001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CAMPOALEGRE%' THEN '41132'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CARTAGENA%' THEN '13001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CERINZA%' THEN '15162'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CHÍA%' THEN '25175'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CHIQUINQUIRÁ%' THEN '15176'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CIRCASIA%' THEN '63190'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%CORRALES%' THEN '15215'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%COTA%' THEN '25214'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%DUITAMA%' THEN '15238'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%EL DOVIO%' THEN '76250'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ENVIGADO%' THEN '05266'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ESPINAL%' THEN '73268'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FACATATIVÁ%' THEN '25269'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FIRAVITOBA%' THEN '15272'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FLORENCIA%' THEN '18001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FLORESTA%' THEN '15276'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FLORIDABLANCA%' THEN '68276'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FUNZA%' THEN '25286'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%FUSAGASUGÁ%' THEN '25290'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GAMEZA%' THEN '15296'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GARAGOA%' THEN '15299'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GIRARDOT%' THEN '25307'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GIRARDOTA%' THEN '05308'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GIRÓN%' THEN '68307'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%GUASCA%' THEN '25322'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%HONDA%' THEN '73349'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%IBAGUÉ%' THEN '73001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ITAGUI%' THEN '05360'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%IZA%' THEN '15362'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA CALERA%' THEN '25377'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA CEJA%' THEN '05376'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA DORADA%' THEN '17380'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA ESTRELLA%' THEN '05380'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA PEÑA%' THEN '25398'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA TEBAIDA%' THEN '63401'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%LA VEGA%' THEN '19397'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MADRID%' THEN '25430'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MANIZALES%' THEN '17001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MANTA%' THEN '25436'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MEDELLÍN%' THEN '05001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MEDINA%' THEN '25438'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MELGAR%' THEN '73449'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MIRAFLORES%' THEN '15455'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MONGUA%' THEN '15464'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MONTERREY%' THEN '85162'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MOSQUERA%' THEN '25473'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%MOTAVITA%' THEN '15476'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%NEIVA%' THEN '41001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%NOBSA%' THEN '15491'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%OIBA%' THEN '68500'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PAIPA%' THEN '15516'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PALMIRA%' THEN '76520'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PAMPLONA%' THEN '54518'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PASCA%' THEN '25535'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PASTO%' THEN '52001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PAUNA%' THEN '15531'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PAZ DE ARIPORO%' THEN '85250'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PEREIRA%' THEN '66001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PESCA%' THEN '15542'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PIEDECUESTA%' THEN '68547'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PITALITO%' THEN '41551'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%PORE%' THEN '85263'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%RESTREPO%' THEN '50606'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%RIOHACHA%' THEN '44001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%RIONEGRO%' THEN '05615'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ROVIRA%' THEN '73624'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SABANETA%' THEN '05631'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SALITRE%' THEN '14129'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SAN ALBERTO%' THEN '20710'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SAN BERNARDO%' THEN '25649'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SAN GIL%' THEN '68679'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SAN LUIS DE PALENQUE%' THEN '85325'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SANTA FE DE ANTIOQUIA%' THEN '05642'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SANTA MARTA%' THEN '47001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SARAVENA%' THEN '81736'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SESQUILÉ%' THEN '25718'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SIBATÉ%' THEN '25740'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SOGAMOSO%' THEN '15759'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SOPÓ%' THEN '25758'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SOPO%' THEN '25758'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SOTAQUIRÁ%' THEN '15761'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SUBACHOQUE%' THEN '25769'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SUAMPAÑA%' THEN '99773'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%SUESCA%' THEN '25772'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TÁMESIS%' THEN '05756'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TENJO%' THEN '25785'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TENZA%' THEN '15778'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TOCANCIPÁ%' THEN '25793'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TOTA%' THEN '15814'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TUNJA%' THEN '15001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TURMEQUÉ%' THEN '15822'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%TUTA%' THEN '15832'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%UBALÁ%' THEN '25797'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%VILLAPINZÓN%' THEN '25862'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%VILLAVICENCIO%' THEN '50001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%VISTA HERMOSA%' THEN '50711'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%YOPAL%' THEN '85001'
        WHEN (SLS.ORDER_CONTACT_TOWN) LIKE '%ZIPAQUIRÁ%' THEN '25899'
        ELSE '11001'
END AS Codigo_ciudad,

SUBSTR(SLS.ORDER_CONTACT_PRIMARY_PHONE,4,10) AS Telefono_principal,
SLS.ORDER_CONTACT_EMAIL,

'SI' AS CLIENTES,
'Activo' AS ESTADO,
'3' AS TIPO_COMPROBANTE,
DENSE_RANK() OVER (ORDER BY SLS.FILE_NUMBER) + '1' - 1 AS Consecutivo,
SLS.ORGANIZATION,
SLS.REFERENCE_DATE as VALUE_DATE,
SLS.PRODUCT,
CASE 
   WHEN NVL(SPP.NET_SOLD_CHARGE_AMT_ITX,'0')!= 0 THEN 'BFEE'
   --WHEN SLS.T_OPERATION_KIND  LIKE '%DELIVERY_OVERHEADS%' THEN 'HFEE'
   WHEN SLS.T_OPERATION_KIND  IN ('DELIVERY_OVERHEADS') THEN 'HFEE' 
   WHEN SLS.T_OPERATION_KIND  IN ('ORDER_OVERHEADS') THEN 'HFEE-CALL'
   WHEN  sls.product_code = ('CAN_INS_30') THEN 'AS-TKF'
   WHEN  sls.product_code = ('CAN_INS_70') THEN 'AS-EL_EVENTO'
   ELSE 'IRT'
END AS Codigo_producto,
'900569193' As Identificacion_vendedor,
CASE 
   WHEN NVL(SPP.NET_SOLD_CHARGE_AMT_ITX,'0')!= 0 THEN ''
   WHEN SLS.T_OPERATION_KIND IN ('DELIVERY_OVERHEADS','ORDER_OVERHEADS') THEN ''
   WHEN  sls.product_code = ('CAN_INS') THEN ''
   WHEN  sls.product_code = ('CAN_INS_30') THEN ''
   WHEN  sls.product_code = ('CAN_INS_70') THEN '901332457'
   ELSE '800181743'
END AS IDENTIFICACION_TERCERO,
CASE 
   WHEN NVL(SPP.NET_SOLD_CHARGE_AMT_ITX,'0')!= 0 THEN '1'
   WHEN SLS.T_OPERATION_KIND  LIKE '%DELIVERY_OVERHEADS%' THEN '1'
   WHEN  sls.product_code = ('CAN_INS') THEN '1'
   WHEN  sls.product_code = ('CAN_INS_30') THEN '1'
   WHEN  sls.product_code = ('CAN_INS_70') THEN '1'
   ELSE ''
END AS Codigo_impuesto,
CASE
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tecnoglass%' THEN 'Otros'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tarjeta VISA%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tarjeta Olimpica%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tarjeta Multiservicios%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tarjeta MASTER%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tarjeta MAESTRO%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tarjeta Intercambios%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tarjeta Exito%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tarjeta ELECTRON%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tarjeta DINERS%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tarjeta de crédito%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tarjeta de credito%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tarjeta Codensa%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Tarjeta AMEX%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Serfinansa%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Saldo FITB%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Saldo Cliente%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Refund requested (for credit card)%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Recaudo Empresario%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Rappi%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Puntos Colombia%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Payvalida%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Pago Salud Total%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Pago Parcial%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Pago en espera%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Otros**%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Olimpica%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Nequi%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%MASTER2%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Junta Directiva%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet VISA Popular%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Visa Nacional%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet VISA Bancolombia Selección%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet VISA Bancolombia%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet VISA AVAL%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet VISA%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Safetypay%' THEN '3'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Rappi%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet OCCIDENTE Visa%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet OCCIDENTE Master%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Nequi VE%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Nequi%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Mastercard Serfinanza%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Mastercard Nacional%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Mastercard Falabella%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Mastercard AvVillas%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet MASTERCARD AVAL%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet MASTERCARD%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet MASTER Bancolombia Selección%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet MASTER Bancolombia%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet DINERS Nacional%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Diners%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Davivienda MasterCard%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Davivienda%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Daviplata%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Dale%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet D24 PSE%' THEN '3'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Colpatria%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet CeroPay%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet BBVA%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet BANCOLOMBIA%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Arma tu vaca%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet AMEX Scotiabank%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet AMEX Nacional%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet AMEX Bancolombia Selección%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet AMEX Bancolombia%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet AMEX%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Internet Addi%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Intercambio Localidad%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Faltante Pago Parcial%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%ELECTRON%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Efectivo Intercambios%' THEN '1'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Efectivo Corporativo%' THEN '1'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Efectivo%' THEN '1'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Devoluciones Efecty%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Descuento Nómina%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Davivienda - Daviplata%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Daviplata%' THEN '4'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Cuentas por Cobrar%' THEN '2'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%CORTESIA%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%CORPORATIVO CRÉDITO %' THEN '2'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%CORPORATIVO CONTADO%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Consignación%' THEN '8'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Codensa%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Cancellation Credit Card%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bonos Softix%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bonos%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%BonoRegaloTB%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%BonoMov%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono Regalo Teatro Mayor%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono Olimpica%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono Maloka%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono Maloka%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono Idartes%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%BONO FOGACOOP%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono FITB 25%%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono FITB 20%%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono FITB 15%%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono Festival Bogotá es Barroco%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono El Cigala%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono Colsubsidio%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono Cliente Barroco%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono Cinemateca%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono Ana Torroja%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono allegro%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Bono 2021%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Billetera Cardenal%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Beneficio 25%%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Beneficio 20%%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Beneficio 15%%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Backoffice%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Aval Pay%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%Atención a clientes%' THEN '104'
    WHEN (SPP.PAYMENT_METHOD) LIKE '%(automatic) Credit note%' THEN '104'
END AS Codigo_pago,
ROUND(
CASE 
   WHEN NVL(SPP.NET_SOLD_CHARGE_AMT_ITX,'0')!= 0 THEN NVL(((SPP.NET_SOLD_CHARGE_AMT_ITX/SLS.ORDER_QUANTITY)/1.19),'0')
   WHEN SLS.T_OPERATION_KIND  LIKE '%DELIVERY_OVERHEADS%' THEN NVL(((SPP.NET_SOLD_AMT_ITX/SLS.ORDER_QUANTITY)/1.19),'0')
   WHEN  sls.product_code = ('CAN_INS') THEN NVL(((SLS.NET_SOLD_TKT_AMT_ITX/SLS.ORDER_QUANTITY)/1.19),'0')
   WHEN  sls.product_code = ('CAN_INS_30') THEN NVL(((SLS.NET_SOLD_TKT_AMT_ITX/SLS.ORDER_QUANTITY)/1.19),'0')
   WHEN  sls.product_code = ('CAN_INS_70') THEN NVL(((SLS.NET_SOLD_TKT_AMT_ITX/SLS.ORDER_QUANTITY)/1.19),'0')
   ELSE NVL((SPP.NET_SOLD_CHARGE_AMT_ITX/SLS.ORDER_QUANTITY),'0') + NVL((SPP.NET_SOLD_AMT_ITX/SLS.ORDER_QUANTITY),'0')
END) AS precio_unitario,
SLS.ORDER_QUANTITY AS,
ROUND
(CASE 
   WHEN NVL(SPP.NET_SOLD_CHARGE_AMT_ITX,'0')!= 0 THEN (NVL(((SPP.NET_SOLD_CHARGE_AMT_ITX/SLS.ORDER_QUANTITY)/1.19),'0')* SLS.ORDER_QUANTITY) 
   WHEN SLS.T_OPERATION_KIND  LIKE '%DELIVERY_OVERHEADS%' THEN (NVL(((SPP.NET_SOLD_AMT_ITX/SLS.ORDER_QUANTITY)/1.19),'0')* SLS.ORDER_QUANTITY)
   WHEN  sls.product_code = ('CAN_INS') THEN (NVL(((SLS.NET_SOLD_TKT_AMT_ITX/SLS.ORDER_QUANTITY)/1.19),'0')*SLS.ORDER_QUANTITY)
   WHEN  sls.product_code = ('CAN_INS_30') THEN (NVL(((SLS.NET_SOLD_TKT_AMT_ITX/SLS.ORDER_QUANTITY)/1.19),'0')*SLS.ORDER_QUANTITY)
   WHEN  sls.product_code = ('CAN_INS_70') THEN (NVL(((SLS.NET_SOLD_TKT_AMT_ITX/SLS.ORDER_QUANTITY)/1.19),'0')*SLS.ORDER_QUANTITY)
   ELSE (NVL((SPP.NET_SOLD_CHARGE_AMT_ITX/SLS.ORDER_QUANTITY),'0') + NVL((SPP.NET_SOLD_AMT_ITX/SLS.ORDER_QUANTITY),'0')*SLS.ORDER_QUANTITY)
END ) AS Total,
sls.file_number AS OBSERVACIONES,
sls.t_operation_type AS Tipo_operacion
--sls.order_number AS Pedido
--,sls.month as month,
--sls.year as year
FROM d_sales_list_sales_v1_0 SLS left join D_SALES_PROD_PAYMENT_V1_0  SPP on (SLS.file_number=SPP.file_number and SLS.t_operation_id=spp.t_operation_id )
left join D_CONTACT_LIST_V1_0 CL on SLS.T_ORDER_CONTACT_ID = CL.T_CONTACT_ID
where --REFERENCE_DATE >= '2024-08-01'
--FILE_NUMBER IN ('','','')
--SUBSTR(sls.order_date,4,7) = '07.2024' AND
--SLS.ORGANIZATION IN  
--and 
--SLS.FILE_NUMBER IN ('11596349') and
SLS.t_operation_kind NOT IN ('PRODUCT_COMPOSITION','CHARGES')
AND SLS.t_operation_type NOT IN ('RESERVATION','RESERVATION_CANCEL')
ORDER BY 
sls.file_number DESC;


"""

In [ ]:
result_df = spark.sql(query)
#result_df.count()
tmp = result_df.filter(col('FILE_NUMBER').isin('9950985', '9773398', '9430896','18525598') )
display(tmp)

IDENTIFICACION,Tipo_identificacion,Tipo,RAZON_SOCIAL,PRIMER_NOMBRE,LASTNAME,Direccion,CODIGO_PAIS,CODIGO_ESTADO,Codigo_ciudad,Telefono_principal,ORDER_CONTACT_EMAIL,CLIENTES,ESTADO,TIPO_COMPROBANTE,Consecutivo,ORGANIZATION,VALUE_DATE,PRODUCT,Codigo_producto,Identificacion_vendedor,IDENTIFICACION_TERCERO,Codigo_impuesto,Codigo_pago,precio_unitario,AS,Total,OBSERVACIONES,Tipo_operacion
142294970,31,Empresa,Nelson,,PIÑERO,null,CO,11,11001,,nelsoncfrt@gmail.com,SI,Activo,3,45550.0,Tren de la Sabana,2023-01-21T00:00:00Z,RECORRIDO ZIPAQUIRA - 2023,IRT,900569193,800181743,,4,70000.0,2,140000.0,9950985,SALE
222222222222,43,Es persona,,Haran,VADIVOO MANIMARAN,null,CO,11,11001,,vadivoomanimaran@gmail.com,SI,Activo,3,44411.0,Tren de la Sabana,2023-01-02T00:00:00Z,RECORRIDO ZIPAQUIRA - 2023,IRT,900569193,800181743,,4,70000.0,5,350000.0,9773398,SALE
222222222222,43,Es persona,,Evan,SULLIVAN,110111 BOGOTA,Co,11,11001,,null,SI,Activo,3,41324.0,Tren de la Sabana,2022-12-01T00:00:00Z,NOCHE DE NAVIDAD EN TREN,IRT,900569193,800181743,,4,55000.0,3,165000.0,9430896,SALE
1031148055,13,Es persona,,Laura Andrea,ARROYAVE MARTÍNEZ,Carrera 28 N,Co,11,11001,3178648506,laura.arroyave08@gmail.com,SI,Activo,3,34884.0,Tuboleta.com,2024-07-26T00:00:00Z,CARGO CALLCENTER,HFEE-CALL,900569193,,,4,3915.0,1,3915.0,18525598,SALE
1031148055,13,Es persona,,Laura Andrea,ARROYAVE MARTÍNEZ,Carrera 28 N,Co,11,11001,3178648506,laura.arroyave08@gmail.com,SI,Activo,3,34884.0,Tuboleta.com,2024-07-26T00:00:00Z,BOLETA DIGITAL (PDF) - $8.000,HFEE,900569193,,1,4,6580.0,1,6580.0,18525598,SALE
1031148055,13,Es persona,,Laura Andrea,ARROYAVE MARTÍNEZ,Carrera 28 N,Co,11,11001,3178648506,laura.arroyave08@gmail.com,SI,Activo,3,34884.0,Tren de la Sabana,2024-07-26T00:00:00Z,VIAJE EN TREN A ZIPAQUIRA 2024,IRT,900569193,800181743,,4,76346.0,2,152692.0,18525598,SALE
1031148055,13,Es persona,,Laura Andrea,ARROYAVE MARTÍNEZ,Carrera 28 N,Co,11,11001,3178648506,laura.arroyave08@gmail.com,SI,Activo,3,34884.0,Tren de la Sabana,2024-07-26T00:00:00Z,VIAJE EN TREN A ZIPAQUIRA 2024,BFEE,900569193,,1,4,4935.0,2,9870.0,18525598,SALE
1031148055,13,Es persona,,Laura Andrea,ARROYAVE MARTÍNEZ,Carrera 28 N,Co,11,11001,3178648506,laura.arroyave08@gmail.com,SI,Activo,3,34884.0,Tren de la Sabana,2024-07-26T00:00:00Z,CANCELACION DEL SEGURO_S_30,AS-TKF,900569193,,1,4,2174.0,1,2174.0,18525598,SALE
1031148055,13,Es persona,,Laura Andrea,ARROYAVE MARTÍNEZ,Carrera 28 N,Co,11,11001,3178648506,laura.arroyave08@gmail.com,SI,Activo,3,34884.0,Tuboleta.com,2024-07-26T00:00:00Z,BOLETA DIGITAL (PDF) - $8.000,HFEE,900569193,,1,4,143.0,1,143.0,18525598,SALE
1031148055,13,Es persona,,Laura Andrea,ARROYAVE MARTÍNEZ,Carrera 28 N,Co,11,11001,3178648506,laura.arroyave08@gmail.com,SI,Activo,3,34884.0,Tren de la Sabana,2024-07-26T00:00:00Z,VIAJE EN TREN A ZIPAQUIRA 2024,BFEE,900569193,,1,4,107.0,2,214.0,18525598,SALE


#### Impresion

In [ ]:
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/Finanzas/Facturacion_Electronica/"

write_table = result_df.coalesce(1) # coalesce indica que la tabla se va a entregar en una sola particion
write_table.write.csv(inpath, header=True, mode="overwrite", sep=';')
 
#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]
 
# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Facturacion_electronica.csv")

True

In [ ]:
# # Guarda el DataFrame en formato Parquet en Azure Blob Storage
# inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/Finanzas/Facturacion_Electronica/"
# # 2024_final
# join_df_coalesced = result_df.coalesce(1)
# #join_df_coalesced = Ds_S3_Sales_2024.coalesce(1)
# join_df_coalesced.write.mode("overwrite").parquet(inpath)
 
# #cambio de nombre
# files=dbutils.fs.ls(inpath)
# output_file= [x for x in files if x.name.startswith("part-")]
 
# # En la misma carpeta de origen
# dbutils.fs.mv(output_file[0].path, f"{inpath}/Facturacion_electronica.parquet")


#### Pruebas

In [ ]:
# display(result_df.filter((col('year')=='2024')&(col('month')=='05')&((col('Codigo_producto')=='AS-EL_EVENTO')|(col('Codigo_producto')=='AS-TKF'))))

In [ ]:
tmp = result_df.select('Codigo_producto').distinct()
display(tmp)

In [ ]:
# from pyspark.sql.functions import col, sum as _sum, first

# # Filtrar las filas según las condiciones
# # filtered_df = sales.filter((col('product_code') == 'CAN_INS_70')|(col('product_code') == 'CAN_INS_30'))
# filtered_df = sales.filter((col('product_code') == 'CAN_INS'))

# # Realizar la suma de la columna 'value' para las filas filtradas
# # suma = filtered_df.groupBy('T_ORGANIZ_ID','year','month').agg((_sum('NET_SOLD_TKT_AMT_ITX').alias('total_value')))
# suma = filtered_df.groupBy('year','month').agg((_sum('NET_SOLD_TKT_AMT_ITX').alias('total_value')))


In [ ]:
#display(suma)

### Prueba de envio de correo

In [ ]:
%pip install yagmail

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached yagmail-0.15.293-py2.py3-none-any.whl (17 kB)
  Using cached premailer-3.10.0-py2.py3-none-any.whl (19 kB)
  Using cached cssutils-2.11.1-py3-none-any.whl (385 kB)
  Using cached cachetools-5.4.0-py3-none-any.whl (9.5 kB)
  Using cached lxml-5.2.2-cp310-cp310-manylinux_2_28_x86_64.whl (5.0 MB)
  Using cached cssselect-1.2.0-py2.py3-none-any.whl (18 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
# import yagmail
# import os
# import tempfile
# from pyspark.sql import SparkSession

# # Inicializar Spark Session
# spark = SparkSession.builder.appName("EmailCSV").getOrCreate()

# # Configuración del correo
# sender_email = 'tenchukev@gmail.com'
# recipient_email = 'kevin.serrano@tuboleta.com'
# app_password = 

# # Asumiendo que ya tienes un DataFrame llamado result_df
# # Si no lo tienes, crea uno de ejemplo:
# #result_df = spark.createDataFrame([(1, "A"), (2, "B"), (3, "C")], ["id", "value"])

# # Convertir a Pandas DataFrame y escribir a CSV
# temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.csv')
# temp_file_path = temp_file.name
# result_df.toPandas().to_csv(temp_file_path, index=False)
# temp_file.close()

# # Inicializar yagmail SMTP
# yag = yagmail.SMTP(sender_email, app_password)

# # Contenido del correo
# subject = 'Reporte generado desde Databricks'
# body = 'Este es el cuerpo del mensaje. El archivo CSV adjunto contiene los datos generados.'

# # Enviar el correo
# try:
#     yag.send(
#         to=recipient_email,
#         subject=subject,
#         contents=body,
#         attachments=temp_file_path
#     )
#     print("Correo enviado exitosamente")
# except Exception as e:
#     print(f"Error al enviar el correo: {e}")

# # Cerrar la conexión SMTP
# yag.close()

# # Eliminar el archivo CSV temporal
# os.remove(temp_file_path)

Error al enviar el correo: (552, b"5.3.4 Your message exceeded Google's message size limits. To view our\n5.3.4 message size guidelines, go to\n5.3.4  https://support.google.com/mail/?p=MaxSizeError 5614622812f47-3db41805cfasm891296b6e.29 - gsmtp", 'tenchukev@gmail.com')


In [ ]:
result_df.count()

159235